[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/morganmcg1/deep-research-bot/blob/main/notebooks/01_simple_agent.ipynb)


# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{fc-london-workshop-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git
```


In [1]:
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git

In [2]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, get_type_hints

### Add your API Keys
We'll need a wandb api key and an exa api key

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

In [24]:
# or

os.environ["WANDB_API_KEY"] = "1a9e11c363f0c16e81ce4f7bafe655183bbc0a64"
os.environ["EXA_API_KEY"] = "6cadb3e7-66c3-4499-b8c2-6af9a460472a"

# or colab secrets:

# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
# os.environ["EXA_API_KEY"] = userdata.get("EXA_API_KEY")

## Model Settings
Define a model to use, as we are going to use tool calling you need a capable model like `Kimi-K2` or `GLM`

In [25]:
class Console(RichConsole):
    def md(self, text):
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

WANDB_ENTITY = "fc-london-llm-team-amunif" # <--- Replace 'YOUR_WANDB_ENTITY' with your actual W&B entity name
WANDB_PROJECT = "london-workshop-2025"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project=f"{WANDB_ENTITY}/{WANDB_PROJECT}")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [26]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

Output()

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: amunif.
weave: View Weave data at https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/01_trace.png?raw=1)

In [27]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
    max_tokens=400,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e80-3be2-720b-98e9-e6b3d906abf3


LLM Response:\nHello! Great question! 😊 An AI agent is a software system that perceives its environment, makes    
decisions, and takes actions to achieve specific goals. Think of it like a digital "entity" that can act           
autonomously — not just following rigid instructions, but adapting and responding intelligently.                   

Here’s how an AI agent works, broken down into key components:                                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs.                                   

 • In a robot: cameras, microphones, temperature sensors.                                                          
 • In a software agent: user inputs, API data, web pages, files.                                                   
 • In a chatbot: your text messages.                                                                               

🧠 Example: A self-driving car "sees" traffic lights using cameras.                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Using the perceived data, the agent processes information and decides what to do next.                             

This can involve:                                                                                                  

 • Rule-based logic (if-then rules)                                                                                
 • Machine learning models (e.g., neural networks)                                                                 
 • Search algorithms (like pathfinding)                                                                            
 • Reinforcement learning (learning from trial and error)                                                          

🧠 Example: A delivery robot decides the fastest route using a map and traffic data.                               

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action that affects its environment.                                                         

 • A robot moves forward.                                                                                          
 • A chatbot sends a reply.                                                                                        
 • A smart thermostat adjusts the temperature.                                                                     

🔁 This action might change the environment, leading to new perceptions — creating a continuous loop.              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             4. Goal-Oriented Behavior                                             

AI agents are usually designed to achieve specific objectives:                                                     

 • Win a game (like AlphaGo)                                                                                       
 • Answer questions accurately (like me!)                                                                          
 • Maximize user engagement (like recommendation systems)                                                          

They often use feedback (rewards or errors) to improve over time.                                                  

─────────────────────────────────────────

Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [28]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.
# if you want to change the name of the tracer use @weave.op(name=whatever)

@weave.op
def call_model(model_name: str, messages: list[dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=400,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e86-ea8d-7101-ba7f-970933efdf7c


Hello! Great question! 😊 An AI agent is a software system that perceives its environment, makes decisions, and    
takes actions to achieve specific goals. Think of it like a digital "assistant" that can think and act             
autonomously, to some degree.                                                                                      

Here’s a simple breakdown of how an AI agent works:                                                                

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs. This could be:                    

 • Text input (like your message to me)                                                                            
 • Sensor data (e.g., from a robot or self-driving car)                                                            
 • Camera images, audio, or web APIs                                                                               

🧠 Example: A chatbot reads your message. A self-driving car sees traffic signs via cameras.                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                          2. Processing & Decision-Making                                          

The agent uses AI models (like machine learning or large language models) to interpret the input and decide what to
do.                                                                                                                

This often involves:                                                                                               

 • Understanding the input (e.g., natural language processing)                                                     
 • Reasoning (e.g., planning steps to solve a problem)                                                             
 • Learning from past experiences (in some agents)                                                                 

🧠 Example: I (this LLM) understand your question about AI agents and plan a helpful response.                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action based on its decision. Actions could be:                                              

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Recommending a product                                                                                          
 • Adjusting a thermostat (in a smart home)                                                                        

🧠 Example: After deciding what to say, I output this text to you.                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                             4. Goal-Oriented Behavior                                             

A good AI agent is designed to achieve a goal, such as:                                                            

 • Answer questions accurately                                                                                     
 • Win a game (like AlphaGo)                                                                                       
 • Navigate a robot to a destination                                                                               
 • Automate customer service               

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/02_nested_trace.png?raw=1)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/function-calling-diagram-steps.png?raw=1)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [29]:
@weave.op
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [30]:
add_numbers(1, 2)

3

In [31]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e8a-82e2-7e1c-a932-5f1625fe050b


TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [32]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [33]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum?"}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e8a-fa8a-7d8d-9709-b51219f54fb1


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-7991e9fe160e4633b6590d9ac879895a',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [34]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

Final Result: 88

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e8c-a13f-7ffd-b474-3d0dbf5fce9b


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [35]:
messages.append(response.model_dump())

In [36]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [37]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum?'},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-7991e9fe160e4633b6590d9ac879895a',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-7991e9fe160e4633b6590d9ac879895a',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [38]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [39]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e8d-5d15-74e0-81ab-f860544f3c08


The sum of your lucky numbers, 77 and 11, is **88**.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [40]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle list[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by list or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [41]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [42]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [43]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [44]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [45]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e90-0191-7de0-acd9-c7660446bdb2


3

### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [46]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [47]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.jamesmartinchef.co.uk/recipes/seabass-en-papiotte-with-salsa-verde/',
            id='https://www.jamesmartinchef.co.uk/recipes/seabass-en-papiotte-with-salsa-verde/',
            title='Seabass en Papillote with Salsa Verde - James Martin Chef',
            score=None,
            published_date='2025-04-25T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text="Find a Recipe:\n\nFind 
Recipes\n\n![](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%200%200'%3E%3C/svg%3
E)\n\n#### Seabass en Papillote with Salsa Verde\n\nSeabass en Papillote! Tender seabass fillets, juicy tomatoes, 
olives and capers — all wrapped up and roasted over hot coals, then finished with a punchy homemade salsa verde. 
Quick, simple, delicious!\n\n[Comfort Food](https://www.jamesmartinchef.co.uk/collections/comfort-food/) \\| 
[Family Favourites](https://www.jamesmartinchef.co.uk/collections/family-favourites/) \\| 
[Fish](https://www.jamesmartinchef.co.uk/collections/fish/) \\| [James Martin's Saturday 
Morning](https://www.jamesmartinchef.co.uk/collections/james-martins-saturday-morning/) \\| [Main 
Courses](https://www.jamesmartinchef.co.uk/collections/main-courses/)\n\n2\n\n#### Ingredients\n\n- 4 IOW tomatoes 
sliced\n- A few capers\n- Stoned olives\n- 2 x 200g seabass fillet \xa0scored\n- Few sprigs\xa0 oregano\n- Drizzle 
of olive oil\n- Salt and pepper\n\nSalsa Verde\n\n- 1 small bunch basil picked and chopped\n- 1 clove garlic, 
peeled and crushed\n- 2 tbs small capers\n- 1 small bunch parsley, chopped\n- 4 anchovies, chopped\n- 1 lemon, 
juice and zest\n- 50ml olive oil\n\n#### Method\n\n[Print Recipe](javascript:window.print();)\n\nHeat a BBQ until 
hot and they coals are white.\n\nPlace a sheet of tinfoil onto the board pop all the ingredients in then place 
another sheet of foil on top. Seal up, pop onto the rack and cook for 4 to 5 minutes.\n\nChop all the ingredients 
together for the verde, open up the parcel then spoon the verde on top.\n\n- 
[Ingredients](https://www.jamesmartinchef.co.uk/recipes/seabass-en-papiotte-with-salsa-verde/#tab-1)\n- 
[Method](https://www.jamesmartinchef.co.uk/recipes/seabass-en-papiotte-with-salsa-verde/#tab-2)\n\n#### 
Ingredients\n\n- 4 IOW tomatoes sliced\n- A few capers\n- Stoned olives\n- 2 x 200g seabass fillet \xa0scored\n- 
Few sprigs\xa0 oregano\n- Drizzle of olive oil\n- Salt and pepper\n\nSalsa Verde\n\n- 1 small bunch basil picked 
and chopped\n- 1 clove garlic, peeled and crushed\n- 2 tbs small capers\n- 1 small bunch parsley, chopped\n- 4 
anchovies, chopped\n- 1 lemon, juice and zest\n- 50ml olive oil\n\n#### Method\n\nHeat a BBQ until hot and they 
coals are white.\n\nPlace a sheet of tinfoil onto the board pop all the ingredients in then place another sheet of 
foil on top. Seal up, pop onto the rack and cook for 4 to 5 minutes.\n\nChop all the ingredients together for the 
verde, open up the parcel then spoon the verde on top.\n\n### Browse My Recipes\n\n[![Beer Can 
Chicken270](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%200%200'%3E%3C/svg%3E)]
(https://www.jamesmartinchef.co.uk/collections/bbq-bites/)\n\n[BBQ 
Bites](https://www.jamesmartinchef.co.uk/collections/bbq-bites/)\n\n[![Knickerbocker Glory with 
Shortbread](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%200%200'%3E%3C/svg%3E)]
(https://www.jamesmartinchef.co.uk/collections/comfort-food/)\n\n[Comfort 
Food](https://www.jamesmartinchef.co.uk/collections/comfort-food/)\n\n[![Salmon with a Champagne 
Sauce](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%200%200'%3E%3C/svg%3E)](http
s://www.jamesmartinchef.co.uk/collections/date-night/)\n\n[Date 
Night](https://www.jamesmartinchef.co.uk/collections/date-night/)\n\n[![Gin Butterm

Let's explore the payload

In [48]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

Find a Recipe:                                                                                                     

Find Recipes                                                                                                       

![](data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%200%200'%3E%3C/svg%3E)         

                                       Seabass en Papillote with Salsa Verde                                       

Seabass en Papillote! Tender seabass fillets, juicy tomatoes, olives and capers — all wrapped up and roasted over  
hot coals, then finished with a punchy homemade salsa verde. Quick, simple, delicious!                             

]8;id=674393;https://www.jamesmartinchef.co.uk/collections/comfort-food/\Comfort Food]8;;\ | ]8;id=592550;https://www.jamesmartinchef.co.uk/collections/family-favourites/\Family Favourites]8;;\ | ]8;id=975213;https://www.jamesmartinchef.co.uk/collections/fish/\Fish]8;;\ | ]8;id=745049;https://www.jamesmartinchef.co.uk/collections/james-martins-saturday-morning/\James Martin's Saturday Morning]8;;\ | ]8;id=749491;https://www.jamesmartinchef.co.uk/collections/main-courses/\Main Courses]8;;\                           

2                                                                                                                  

                                                    Ingredients                                                    

 • 4 IOW tomatoes sliced                                                                                           
 • A few capers                                                                                                    
 • Stoned olives                                                                                                   
 • 2 x 200g seabass fillet  scored                                                                                 
 • Few sprigs  oregano                                                                                             
 • Drizzle of olive oil                                                                                            
 • Salt and pepper                                                                                                 

Salsa Verde                                                                                                        

 • 1 small bunch basil picked and chopped                                                                          
 • 1 clove garlic, peeled and crushed                                                                              
 • 2 tbs small capers                                                                                              
 • 1 small bunch parsley, chopped                                                                                  
 • 4 anchovies, chopped                                                                                            
 • 1 lemon, juice and zest                                                                                         
 • 50ml olive oil                                                                                                  

                                                      Method                                                       

[Print Recipe](javascript:window.print();)                                                                         

Heat a BBQ until hot and they coals are white.                                                                     

Place a sheet of tinfoil onto the board pop all the ingredients in then place another sheet of foil on top. Seal   
up, pop onto the rack and cook for 4 to 5 minutes.                                                                 

Chop all the ingredients together for the verde, open up the parcel then spoon the verde on top.                   

 • ]8;id=769908;https://www.jamesmartinchef.co.uk/recipes/seabass-en-papiotte-

In [49]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output



In [50]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [51]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e92-6cfe-7f25-b04c-98dc9f6cd201


[
    {
        'title': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man's World ®",
        'text': '[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\n\n# Chilean Sea 
Bass Recipe\n\nBy [Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\n\n[Jump to Recipe
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\n\nReader Rating\n\nTotal Time
10 minutesmins\n\nServings 4servings\n\n_This post may contain affiliate links. See my [disclosure 
policy](https://thebigmansworld.com/privacy-policy/)._\n\n_This pan seared **Chilean sea bass recipe** comes 
together in under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and delicious._\n\n**★★★★★ 
REVIEW**\n\n_“OMG, this recipe was the easiest and the most delicious! My sister had never had sea bass before, so 
I made it for both of us. I did get it without the skin, and it was absolutely amazing! If you haven’t tried this, 
please use this recipe, you won’t regret it, the absolute best!”_ – Pam\n\nTable of Contents\n\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\n3. [Chilean sea bass cooking
tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\n5. [Seared Chilean Sea Bass Recipe (Recipe 
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\n\nFriends, if you are looking 
for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my quick, 
pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or elegant
events, making it at home is SO easy and tastes just as good- if not better!\n\nIt’s so tender, flaky, and 
naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to make 
(prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which lets
the natural flavor of the Chilean sea bass shine.\n\nIf you love elegant seafood dinners, try my **[parmesan 
crusted tilapia](https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **[grilled tuna 
steak](https://thebigmansworld.com/grilled-tuna-steak/)**, **[roasted 
branzino](https://thebigmansworld.com/branzino-recipe/)**, and 
**[rockfish](https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\n\nWhat is Chilean Sea 
Bass?\n\nChilean sea bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to 
the cold sub-Antarctic regions of the Pacific, southern Atlantic, and Indian Oceans.\n\nIt’s high in protein and 
super low in carbs. It’s famous for its silky, buttery texture and mild flavor (no fishy taste). It’s considered a 
premium fish due as its hard to overcook and super versatile.\n\n## Key Ingredients\n\n- **Chilean sea bass 
fillets**. Either fresh or frozen fillets.\xa0Because of the slightly higher price tag, I prefer to get the fillets
from a fishmonger over a grocery store. When choosing sea bass, opt for fillets that have no strong odors, no dry 
spots, and are firm to touch.\n- **Kosher salt and black pepper**. To taste.\n- **Unsalted butter**. Pan-frying 
white fish fillets in butter takes the dish’s flavor to another level and cooks the fish perfectly. Olive oil will 
also work.\n- **Garlic**. Freshly minced garlic, not the jarred kind!\n- **Lemon**. Fresh lemons to squeeze over 
the freshly cooked fish.\n\n## How to cook Chilean sea bass\n\n_This is an overview with step-by-step photos. Full 
ingredients & instructions are in the [recipe card 
below](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)._\n\n**Step 1-** Pat dry 
the fish filets wi

In [52]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e92-87ab-76d6-af1a-25e6eb552876


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-91445d0b92de46bb9bfffff9c229b2d3',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [53]:
def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[Callable]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    if not tool_calls:
        return messages
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args) # doesn't handle async

        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"

        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"

        console.print(Panel(panel_content, border_style="cyan"))

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [54]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e93-3463-7c32-a3b5-62c0e1bd13e3


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e93-3879-72b1-88d0-5c6a5c1d1725


────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking seabass is a simple and elegant process that can be done in multiple ways, including pan-searing, baking,  
or pan-roasting. Here’s a comprehensive guide based on the search results:                                         

                                   1. Pan-Seared Sea Bass (Quick & Easy Method)                                    

This method delivers a crispy exterior and tender, flaky interior in under 10–15 minutes.                          

Ingredients:                                                                                                       

 • Sea bass fillets (preferably with skin)                                                                         
 • Salt and black pepper                                                                                           
 • Butter or avocado oil                                                                                           
 • Garlic (optional)                                                                                               
 • Lemon (for finishing)                                                                                           

Steps:                                                                                                             

 1 Prep the fish: Pat the fillets dry with a paper towel to ensure a good sear.                                    
 2 Season: Sprinkle both sides with salt and pepper.                                                               
 3 Heat the pan: Use a non-stick or stainless steel skillet over medium-high heat. Add butter or oil until hot (but
   not smoking).                                                                                                   
 4 Sear skin-side down: Place the fillet skin-side down and cook for 3–5 minutes until golden and crispy. Do not   
   move it to allow proper searing.                                                                                
 5 Flip and finish: Turn the fillet and cook for another 1–3 minutes, depending on thickness.                      
 6 Finish: Remove from the pan, drizzle with pan juices, and squeeze fresh lemon on top.                           

Tip: For thicker fillets, transfer the pan to a 400°F (200°C) oven for 3–5 minutes after searing to ensure even    
cooking.                                                                                                           

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      2. Baked Sea Bass (Simple & Hands-Off)                                       

Perfect for a low-carb, keto-friendly meal.                                                                        

Ingredients:                                                                                                       

 • Sea bass fillets                                                                                                
 • Melted butter                                                                                                   
 • Salt, pepper, paprika (optional)                                                                                
 • Lemon                                                                                                           

Steps:                                                                                                             

 1 Preheat oven to 400°F (200°C).                                                                                  
 2 **

Let's wrap this in a function:

In [55]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL,
        messages=messages,
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))

    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [56]:
result = research("What are the most popular pokemons?")
console.md(result)

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e94-0e3b-7984-b045-3d31bd3a6478


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the search results, the most popular Pokémon vary depending on the context (e.g., fan votes, trading card 
popularity, or regional preferences), but several names consistently emerge as fan favorites:                      

                              Top Popular Pokémon According to Fan Votes and Surveys                               

 1 Charizard                                                                                                       
    • Frequently ranked as the most popular Pokémon in multiple surveys, including a 2019 Reddit poll of over      
      52,000 users where it secured the #1 spot with 1,107 votes. It is praised for its iconic design, strength,   
      and appearances in both games and anime.                                                                     
    • A highly sought-after Pokémon in the Trading Card Game (TCG), with first-edition holographic Charizard cards 
      selling for tens of thousands of dollars.                                                                    
 2 Gengar                                                                                                          
    • Ranked #2 in the same Reddit poll with 1,056 votes. Its popularity has grown over time due to its spooky yet 
      charming design, making it a favorite during Halloween events and in competitive play.                       
 3 Arcanine                                                                                                        
    • Came in at #3 in the Reddit survey with 923 votes. Known for its majestic appearance and legendary speed     
      (said to run over 6,200 miles in a day), Arcanine has a strong nostalgic following.                          
 4 Bulbasaur                                                                                                       
    • Ranked #4 with 710 votes. Despite not being the strongest starter, Bulbasaur is beloved for its cute design  
      and frequent appearances in media, especially as part of Ash Ketchum’s team in the anime.                    
 5 Umbreon                                                                                                         
    • Ranked #6 in the Reddit poll and is particularly popular among fans of the TCG and Pokémon GO. It is often   
      associated with "moon" themes and has a sleek, dark aesthetic that appeals to many.                          
 6 Lucario                                                                                                         
    • Ranked #7 in the Reddit poll and placed #2 in The Pokémon

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/04_pokedex.png?raw=1)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/05_agent.png?raw=1)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [57]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]

    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM,
            messages=messages,
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=tools, tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [58]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e94-e268-79e7-ae12-e2cfb318c16a


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "largest most populous cities European Union population 2024 2025", "num_results": 5}           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/4 ───────────────────────────────────────────────

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Now I need to analyze the search results to identify the 2 major EU cities and their populations, then calculate   
the sum.                                                                                                           

Looking at the data, I can see different sources provide different rankings based on different methodologies:      

 1 The Statista data shows largest European cities by urban agglomeration (which includes some non-EU countries)   
 2 The Wikipedia article on "List of cities in the European Union by population within city limits" shows only EU  
   cities                                                                                                          
 3 The World Atlas data shows largest European cities by metropolitan area population                              

For EU cities specifically, I should focus on the EU member states. Looking at the sources:                        

From the Wikipedia article "List of cities in the European Union by population within city limits", the top 2 are: 

 1 Berlin, Germany: 3,897,145                                                                                      
 2 Madrid, Spain: 3,460,771                                                                                        

However, I should be careful because different sources use different methodologies. The Statista data shows larger 
numbers because it uses urban agglomeration, but includes some non-EU cities.                                      

Let me check what the most accurate definition would be for "major EU cities." The user is asking for the 2 major  
EU cities, so I should look for the 2 largest cities within the European Union.                                    

From the EU-specific data, the top 2 cities by population within city limits are:                                  

 1 Berlin, Germany: 3,897,145                                                                                      
 2 Madrid, Spain: 3,460,771                                                                                        

However, I should also consider that Paris might have a larger metropolitan area even if its city proper is        
smaller. From the World Atlas data, Paris shows 11,215,000 (metropolitan area) which would make it much larger than
Berlin's city proper.                                                                                              

But the user asked for "cities" and didn't specify the methodology. Let me use the most commonly accepted          
definition which would be city proper or administrative boundaries for consistency.                                

I'll calculate the

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [59]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: list[dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [60]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: list[Callable], state_class: BaseModel = AgentState):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
        self.state_class = state_class

    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name,
                messages=messages,
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return self.state_class(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10, **state_kwargs: Any) -> AgentState:
        state = self.state_class(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}], **state_kwargs)
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state



![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/07_simple_agent.png?raw=1)

In [61]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers],
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e96-acaf-74fc-a51b-deb44a81b897


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash's first 3 Pokémon and their weights", "num_results": 5}                                    │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "weight of Ash's Pikachu Bulbasaur and Charmander", "num_results": 5}                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 3/3 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 6, "b": 6.9}                                                                                        │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final response: None


In [62]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for item, result in enumerate(search_results.results):
        console.print(f"Refining result {item+1}")
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [63]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")

weave: 🍩 https://wandb.ai/fc-london-llm-team-amunif/london-workshop-2025/r/call/019a4e98-6cdd-7f8e-888f-07bb839c9b4a


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

Output()

Refining result 1

Refining result 2

...

Refining result 4

Refining result 5

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search_and_refine                                                                             │
│                                                                                                                 │
│ Args: {"query": "Ash Ketchum first three Pokémon names and weights", "num_results": 5}                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

Final response: Ash Ketchum's first three Pokémon are widely recognized as **Pikachu**, **Bulbasaur**, and **Charmander**. Based on official Pokémon species data:

1. **Pikachu** – Weight: **6.0 kg**  
2. **Bulbasaur** – Weight: **6.9 kg**  
3. **Charmander** – Weight: **8.5 kg**  

To find the combined weight:

$$
6.0\,\text{kg} + 6.9\,\text{kg} + 8.5\,\text{kg} = 21.4\,\text{kg}
$$

### ✅ Combined Weight: **21.4 kg**

**Sources:**  
- [Bulbapedia - Ash's Bulbasaur](https://bulbapedia.bulbagarden.net/wiki/Ash%27s_Bulbasaur)  
- [Character Profile Wikia - Ash Ketchum](https://characterprofile.fandom.com/wiki/Ash_Ketchum)  
- [Bulbapedia - Ash Ketchum](https://bulbapedia.bulbagarden.net/wiki/Ash_Ketchum)


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.

## Evaluating our SimpleAgent

In [64]:
# Run this in colab

!git clone https://github.com/morganmcg1/deep-research-bot/
!mv deep-research-bot/data /data

Cloning into 'deep-research-bot'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 609 (delta 104), reused 100 (delta 78), pack-reused 448 (from 1)
Receiving objects: 100% (609/609), 3.35 MiB | 10.35 MiB/s, done.
Resolving deltas: 100% (358/358), done.


In [65]:
import sys
from functools import partial
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [66]:
from deep_research_bot.evaluation.eval import run_evaluation
from deep_research_bot.evaluation.eval_config import EvalConfig

MAX_TURNS=5

agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers],
)

eval_config = EvalConfig(
    evaluation_name=f"SimpleAgent_max-turns-{MAX_TURNS}_{agent.model_name.split('/')[-1]}",
    trials=2,
    limit=20,
    judge_model="deepseek-ai/DeepSeek-R1-0528",
    weave_parallelism=4,
    queries=project_root / "data/prompt_data/query.jsonl",
    reference=project_root / "data/test_data/cleaned_data/reference.jsonl",
    criteria=project_root / "data/criteria_data/criteria.jsonl",
)

results = await run_evaluation(
    eval_config=eval_config,
    agent_callable=partial(agent.run, max_turns=MAX_TURNS),  # <- partial to limit the number of agent turns
)
results

ValueError: OPENAI_API_KEY is not set, please set it in the .env file.